In [ ]:
# ! pip install Bottleneck

In [ ]:
import numpy as np

import torch
import torch.nn.functional as F
from torch import nn, optim

from scipy import sparse
from scipy.sparse import load_npz

import bottleneck as bn

import random
from copy import deepcopy
import time

from tqdm.notebook import tqdm

In [ ]:
seed = 1337
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device("cuda:0")

In [ ]:
dataset = # dataset name

if dataset not in ['ml20m', 'netflix', 'msd', 'amazon-book', 'yelp2018', 'gowalla']:
    raise

In [ ]:
if dataset == 'ml20m':
    params = {'bs_min': 6.79, 'bs_vel': 0.24,
              '-nu': 0.3,
              'γ_st %': -0.28, 'γ_vel': 0.0389,
              'n_hid': 2048, 'n_epochs': 100}
    
if dataset == 'netflix':
    params = {'bs_min': 10, 'bs_vel': 0.166,
              '-nu': 0.175,
              'γ_st %': 1.4, 'γ_vel': 0.0325,
              'n_hid': 2048, 'n_epochs': 100}

if dataset == 'msd':
    params = {'bs_min': 16, 'bs_vel': 0.2,
              '-nu': 0.1,
              'γ_st %': 1.2, 'γ_vel': 0.05,
              'n_hid': 4096, 'n_epochs': 100}

if dataset == 'amazon-book':
    params = {'bs_min': 13.899, 'bs_vel': 0.271,
              '-nu': 0.302,
              'γ_st %': -0.169, 'γ_vel': 0.009,
              'n_hid': 4096, 'n_epochs': 100}

if dataset == 'yelp2018':
    params = {'bs_min': 16, 'bs_vel': 0.15,
              '-nu': 0.096,
              'γ_st %': 0.05, 'γ_vel': 0.0026,
              'n_hid': 1024, 'n_epochs': 100}

if dataset == 'gowalla':
    params = {'bs_min': 14.66, 'bs_vel': 0.317,
              '-nu': 0.450,
              'γ_st %': -2.079, 'γ_vel': 0.0427,
              'n_hid': 4096, 'n_epochs': 100}

In [ ]:
train_data = load_npz('data/' + dataset + '/train_data.npz')
valid_in_data = load_npz('data/' + dataset + '/valid_in_data.npz')
valid_out_data = load_npz('data/' + dataset + '/valid_out_data.npz')
test_in_data = load_npz('data/' + dataset + '/test_in_data.npz')
test_out_data = load_npz('data/' + dataset + '/test_out_data.npz')

n_users, n_items = train_data.shape

# Metrics

In [ ]:
def ndcg(X_pred, heldout_batch, k=100):
    '''
    normalized discounted cumulative gain@k for binary relevance
    ASSUMPTIONS: all the 0's in heldout_data indicate 0 relevance
    '''
    batch_users = X_pred.shape[0]
    idx_topk_part = bn.argpartition(-X_pred, k, axis=1)
    topk_part = X_pred[np.arange(batch_users)[:, np.newaxis],
                       idx_topk_part[:, :k]]
    idx_part = np.argsort(-topk_part, axis=1)
    # X_pred[np.arange(batch_users)[:, np.newaxis], idx_topk] is the sorted
    # topk predicted score
    idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis], idx_part]
    # build the discount template
    tp = 1. / np.log2(np.arange(2, k + 2))

    DCG = (heldout_batch[np.arange(batch_users)[:, np.newaxis],
                         idx_topk].toarray() * tp).sum(axis=1)
    IDCG = np.array([(tp[:min(n, k)]).sum()
                     for n in heldout_batch.getnnz(axis=1)])
    return DCG / IDCG


def recall(X_pred, heldout_batch, k=100):
    batch_users = X_pred.shape[0]

    idx = bn.argpartition(-X_pred, k, axis=1)
    X_pred_binary = np.zeros_like(X_pred, dtype=bool)
    X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx[:, :k]] = True

    X_true_binary = (heldout_batch > 0).toarray()
    tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(
        np.float32)
    recall = tmp / np.minimum(k, X_true_binary.sum(axis=1))
    return recall

In [ ]:
if dataset in ['ml20m', 'netflix', 'msd']:
    metrics = [{'metric': ndcg, 'k': 100}]
    test_metrics = [{'metric': ndcg, 'k': 100}, {'metric': recall, 'k': 20}, {'metric': recall, 'k': 50}]
    
if dataset in ['amazon-book', 'yelp2018', 'gowalla']:
    metrics = [{'metric': ndcg, 'k': 20}]
    test_metrics = [{'metric': ndcg, 'k': 20}, {'metric': recall, 'k': 20}]




# Batch generator

In [ ]:
def generate(batch_size, device, data_in, data_out=None, shuffle=False, samples_perc_per_epoch=1):
    assert 0 < samples_perc_per_epoch <= 1
    
    total_samples = data_in.shape[0]
    samples_per_epoch = int(total_samples * samples_perc_per_epoch)
    
    if shuffle:
        idxlist = np.arange(total_samples)
        np.random.shuffle(idxlist)
        idxlist = idxlist[:samples_per_epoch]
    else:
        idxlist = np.arange(samples_per_epoch)
    
    for st_idx in range(0, samples_per_epoch, batch_size):
        end_idx = min(st_idx + batch_size, samples_per_epoch)
        idx = idxlist[st_idx:end_idx]

        yield Batch(device, idx, data_in, data_out)


class Batch:
    def __init__(self, device, idx, data_in, data_out=None):
        self._device = device
        self._idx = idx
        self._data_in = data_in
        self._data_out = data_out
    
    def get_idx(self):
        return self._idx
    
    def get_idx_to_dev(self):
        return torch.LongTensor(self.get_idx()).to(self._device)
        
    def get_ratings(self, is_out=False):
        data = self._data_out if is_out else self._data_in
        return data[self._idx]
    
    def get_ratings_to_dev(self, is_out=False):
        return torch.Tensor(
            self.get_ratings(is_out).toarray()
        ).to(self._device)

# Model

In [ ]:
class RBM(nn.Module):

    def __init__(self, n_vis, n_hid, γ, nu, k=1):
        super(RBM, self).__init__()
        self.b = nn.Parameter(torch.randn(1, n_vis))
        self.c = nn.Parameter(torch.randn(1, n_hid))
        self.W = nn.Parameter(torch.randn(n_hid, n_vis))
        
        self.k = k
        self.γ = γ
        self.nu = nu
        self.k = k
        
        self.Z = np.mean( np.power(train_data.sum(1), nu) )
        
        for (_, p) in self.named_parameters():
            p.data.normal_(0, 0.02)
            p.grad = p.data.clone()
            
    def beta(self, x):
        return self.γ * (x.sum(1, keepdim=True)).pow(self.nu) / self.Z
        
    def visible_to_hidden(self, beta, v, mean=False):
        p = torch.sigmoid(
            beta * v @ self.W.data.T + beta * self.c.data
        )
        return p if mean else p.bernoulli()

    def hidden_to_visible(self, beta, h, mean=False):
        p = torch.sigmoid(
            beta * h @ self.W.data + beta * self.b.data
        )
        return p if mean else p.bernoulli()
    
    def f_v(self, beta, v):
        res = F.softplus(beta * v @ self.W.T + beta * self.c).sum(1).mean()
        res += (beta * v @ self.b.T).mean()
        return -res
    
    def forward(self, beta, v_data, v_gibbs):
        return self.f_v(beta, v_data) - self.f_v(beta, v_gibbs)

    def sample(self, beta, v_data):
        v_gibbs = v_data.clone()
        
        for _ in range(self.k):
            v_gibbs = (v_gibbs * 0.5).bernoulli()

            h = self.visible_to_hidden(beta, v_gibbs)
            v_gibbs = self.hidden_to_visible(beta, h)
        return v_gibbs

# Prediction methods

In [ ]:
def evaluate(model, method, data_in, data_out, metrics, samples_perc_per_epoch=1, batch_size=500):
    metrics = deepcopy(metrics)
    model.eval()
    time_sum = 0
    
    for m in metrics:
        m['score'] = []
    
    for batch in generate(batch_size=batch_size,
                          device=device,
                          data_in=data_in,
                          data_out=data_out,
                          samples_perc_per_epoch=samples_perc_per_epoch
                         ):
        
        ratings_in_dev = batch.get_ratings_to_dev()
        ratings_in = batch.get_ratings()
        ratings_out = batch.get_ratings(is_out=True)
        if (ratings_out.sum(1) < 1).sum() > 0:
            ids = np.array(ratings_out.sum(1) > 0)[:,0]
            ratings_out = ratings_out[ids]
            ratings_in = ratings_in[ids]
            ratings_in_dev = ratings_in_dev[torch.tensor(ids).to(device)]
    
        time_start = time.time()        
        ratings_pred = method(model, ratings_in_dev).cpu().numpy()
            
        time_end = time.time()
        time_sum += time_end - time_start
            
        if not (data_in is data_out):
            ratings_pred[batch.get_ratings().nonzero()] = -np.inf
            
        for m in metrics:
            m['score'].append(m['metric'](ratings_pred, ratings_out, k=m['k']))

    for m in metrics:
        m['score'] = np.concatenate(m['score']).mean()
        
    print(time_sum)
    return [x['score'] for x in metrics]


def gaussian_probit_approx(model, x):
    beta = model.beta(x)
    p = model.visible_to_hidden(beta, x, mean=True)
    d = beta * p * (1 - p) * beta
    cov = d @ model.W.data.pow(2)
    B = (1 + cov * torch.pi / 8).sqrt()
    return ((beta * p @ model.W.data + beta * model.b.data) / B)

    
def first_order_approx(model, x):
    beta = model.beta(x)
    h = model.visible_to_hidden(beta, x, mean=True)
    return model.hidden_to_visible(beta, h, mean=True)


def MC_approx(model, x, N):
    beta = model.beta(x)
    h = model.visible_to_hidden(beta, (x * 0.9).bernoulli())
    ratings_pred = model.hidden_to_visible(beta, h, mean=True)
    for _ in range(N):
        h = model.visible_to_hidden(beta, (x * 0.9).bernoulli())
        ratings_pred += model.hidden_to_visible(beta, h, mean=True)
    return ratings_pred


def density_based_pred(model, x):
    beta = model.beta(x)
    W, b, c = model.W.data, model.b.data, model.c.data
    res = F.softplus( (beta * x @ W.T + beta * c)[:,None,:] + ((beta * torch.ones_like(x))[:,:,None] * W.T[None,:,:]) ).sum(dim=-1)
    res -= F.softplus( (beta * x @ W.T + beta * c)[:,None,:] + ((beta * torch.zeros_like(x))[:,:,None] * W.T[None,:,:]) ).sum(dim=-1)
    res += ((beta * torch.ones_like(x))[:,:,None] * b.T[None,:,:])[:,:,0]
    res -= ((beta * torch.zeros_like(x))[:,:,None] * b.T[None,:,:])[:,:,0]
    return res

# Training

In [ ]:
params['lr'] = 0.0005
params['bs_max'] = 4096

bs_min, bs_vel, bs_max = params['bs_min'], params['bs_vel'], params['bs_max']
γ_start, γ_vel = params['γ_st %'], params['γ_vel']

print(params)


best_score = -np.inf
best_epoch = -1
valid_scores = []

model_best = RBM(n_items, params['n_hid'], None, -params['-nu']).to(device)
model      = RBM(n_items, params['n_hid'], None, -params['-nu']).to(device)
opt = optim.Adam(model.parameters(), params['lr'])

model.train()

for epoch in range(params['n_epochs']):

    batch_size = min(2**(int(epoch * bs_vel + np.log2(bs_min))), bs_max)
    model.γ = 1 / (epoch * γ_vel + 2**γ_start)

    for batch in generate(batch_size=batch_size, device=device, data_in=train_data, shuffle=True):
        v_data = batch.get_ratings_to_dev()
        beta = model.beta(v_data)
        v_gibbs = model.sample(beta, v_data)
        opt.zero_grad()
        loss = model(beta, v_data, v_gibbs)
        loss.backward()            
        opt.step()


    score = evaluate(model, gaussian_probit_approx, valid_in_data, valid_out_data, metrics, samples_perc_per_epoch=1)[0]
    valid_scores.append(score)

    if score > best_score:
        best_score = score
        best_epoch = epoch

        model_best.load_state_dict(deepcopy(model.state_dict()))
        model_best.γ = model.γ

    print(f'epoch {epoch} | valid ndcg@100: {score:.4f} | best: {best_score:.4f} | ' + 
          f'batch size: {batch_size} | ' +
          f'temp: {1 / model.γ:.3f}')

    if epoch - best_epoch > 10:
        break

# Prediction

In [ ]:



print('Gaussian-probit approx')
final_scores = evaluate(model_best, gaussian_probit_approx, test_in_data, test_out_data, test_metrics)
for metric, score in zip(test_metrics, final_scores):
    print(f"{metric['metric'].__name__}@{metric['k']}:\t{score:.4f}")

print('First-order approx')
final_scores = evaluate(model_best, first_order_approx, test_in_data, test_out_data, test_metrics)
for metric, score in zip(test_metrics, final_scores):
    print(f"{metric['metric'].__name__}@{metric['k']}:\t{score:.4f}")

print('MC approx, N=100')
final_scores = evaluate(model_best, lambda model, x: MC_approx(model, x, 100), test_in_data, test_out_data, test_metrics)
for metric, score in zip(test_metrics, final_scores):
    print(f"{metric['metric'].__name__}@{metric['k']}:\t{score:.4f}")
    
print('MC approx, N=1000')
final_scores = evaluate(model_best, lambda model, x: MC_approx(model, x, 1000), test_in_data, test_out_data, test_metrics)
for metric, score in zip(test_metrics, final_scores):
    print(f"{metric['metric'].__name__}@{metric['k']}:\t{score:.4f}")
    
print('Density-based prediction')
final_scores = evaluate(model_best, density_based_pred, test_in_data, test_out_data, test_metrics, batch_size=5)
for metric, score in zip(test_metrics, final_scores):
    print(f"{metric['metric'].__name__}@{metric['k']}:\t{score:.4f}")
